# Generating Slippy Map Tiles with GDAL

## Objective

When previewing large rasters, it's helpful to have people be able
to zoom around, but this is difficult without hosting tiles somewhere.

## Process

1. Procure your data.
2. Pick your color palette from the [GRASS Wiki](https://grasswiki.osgeo.org/wiki/Color_tables) and procure the text file from the GRASS SVN repository
3. Use `gdaldem` to recolor the raster
4. Use `gdal2tiles.py` to slice up the raster.
5. Serve the directory somewhere where you can serve static files and load the tiles in your map.

## Use `global_dem.tif` from the InVEST Sample Data

This is a nice file to demo - not too fine to slow down tiling but still al global dataset.

In [ ]:
! cp ~/workspace/natcap/invest-sample-data/Base_Data/global_dem.tif .

## SRTM Elevation Colormap

![SRTM Colormap](https://grasswiki.osgeo.org/w/images/Colortable_srtm.png "SRTM Colormap")

In [ ]:
! wget https://svn.osgeo.org/grass/grass/branches/releasebranch_6_4/lib/gis/colors/srtm -O srtm.gcolors
! cat srtm.gcolors

## Process the Raster into Tiles

In [ ]:
! gdaldem color-relief global_dem.tif srtm.gcolors global_dem_colorized.tif

In [ ]:
! gdal2tiles.py --xyz -r near --zoom=1-6 --processes=3 global_dem_colorized.tif tiles/

## Preview the Tiles with Leaflet

In [ ]:
import ipyleaflet
from ipyleaflet import Map, LocalTileLayer, basemap_to_tiles, basemaps

chosen_basemap = basemap_to_tiles(basemaps.OpenStreetMap.Mapnik)
chosen_basemap.name = '(basemap) OpenStreetMap Mapnik'

m = Map(
    basemap=chosen_basemap,
    center=(0,0),
    zoom=2,  # zoom level for most of the globedisplay(m)
)

tiles_layer = LocalTileLayer(
    path='tiles/{z}/{x}/{y}.png',
    name='InVEST Sample DAta',
    show_loading=True,
    attribution='InVEST'
)
m.add_layer(tiles_layer)

m

## Serve the map tiles somewhere

At this point, the `tiles/` directory is just a directory of files that can
be copied to your webserver of choice, such as a public google cloud bucket,
or even github pages if the tile cache doesn't exceed github's repository
limits.